In [1]:
from model import *
from algorithms import *
from utils import *
from trainer import *
from env import * 
from logger import *

from IPython.display import clear_output

import torch
import os

env = MaxStepContinuousCartPoleEnv()

actor_model = NNActor()
critic_model = NNCritic()

expert_model = NNActor()
expert_model.load_state_dict(torch.load("data/model/expert_training/5/12700.pth"))
expert_model.eval()

trainer_config = dict(
    actor_optimizer_name = "Adam",
    actor_optimizer_kwargs = dict(lr = 1e-6, weight_decay = 1e-4),
    critic_optimizer_name = "Adam",
    critic_optimizer_kwargs = dict(lr = 1e-4),
    discount = 0.99, 
    tau = 0.005, 
    noise = 0.2, 
    actor_update_period = 2,
    batch_size = 24,
    )

algorithm_config = dict(
    replay_buffer_size = int(1e6),
    noise = 0.2)

replay_buffer = ReplayBuffer(algorithm_config['replay_buffer_size'])

trainer = SLRL(
    actor_model,
    critic_model,
    replay_buffer = replay_buffer,
    only_rl = True,
    device = 'cpu',
    **trainer_config)

logger = WandbLogger(name = "Training Agent [RL]", id = 6, config_dict = dict(trainer = trainer_config, algorithm = algorithm_config))

algo = IIL_algorithm(env, trainer, expert_model, replay_buffer, noise = algorithm_config['noise'], logger = logger)

while True:
    algo.run(10)
    clear_output(wait=True)

In [ ]:
len(replay_buffer)